------------

In [1]:
{-# LANGUAGE OverloadedStrings #-}

import Data.Aeson
import qualified Data.ByteString.Lazy as B
import Data.Text.Prettyprint.Doc

-- Define a data type to represent the structure of the JSON
data Location = Location
  { loc_name :: String
  , loc_class :: String
  , loc_directory :: String
  , loc_remote_node :: String
  , loc_remote_login :: String
  , loc_remote_pwd :: String
  , loc_remote_port :: Int
  , loc_db_name :: String
  , loc_db_user :: String
  , loc_description :: String
  } deriving (Show)

instance Pretty Location where
  pretty loc =
    vsep
      [ "Location {"
      , indent 2 ("loc_name = " <> pretty (loc_name loc))
      , indent 2 ("loc_class = " <> pretty (loc_class loc))
      , indent 2 ("loc_directory = " <> pretty (loc_directory loc))
      , indent 2 ("loc_remote_node = " <> pretty (loc_remote_node loc))
      , indent 2 ("loc_remote_login = " <> pretty (loc_remote_login loc))
      , indent 2 ("loc_remote_pwd = " <> pretty (loc_remote_pwd loc))
      , indent 2 ("loc_remote_port = " <> pretty (loc_remote_port loc))
      , indent 2 ("loc_db_name = " <> pretty (loc_db_name loc))
      , indent 2 ("loc_db_user = " <> pretty (loc_db_user loc))
      , indent 2 ("loc_description = " <> pretty (loc_description loc))
      , "}"
      ]

-- Define an instance to decode JSON into the Location data type
instance FromJSON Location where
  parseJSON = withObject "Location" $ \v -> Location
    <$> v .: "loc_name"
    <*> v .: "loc_class"
    <*> v .: "loc_directory"
    <*> v .: "loc_remote_node"
    <*> v .: "loc_remote_login"
    <*> v .: "loc_remote_pwd"
    <*> v .: "loc_remote_port"
    <*> v .: "loc_db_name"
    <*> v .: "loc_db_user"
    <*> v .: "loc_description"

-- Read JSON data from the file
jsonData <- B.readFile "data.json"
  
-- Parse JSON using eitherDecode function
let parsedData = eitherDecode jsonData :: Either String [Location]


In [2]:
data Location' = Location'
  { loc_name' :: String
  , loc_class' :: String
  , loc_directory'::String
  , loc_remote_node' :: String  -- agent used 
  , loc_remote_login' :: String -- user agent 
  , loc_remote_port' :: Int     -- agent port
  , loc_db_user' :: String
  , loc_description' :: String
  , db_node_name :: String      -- infra location where data is stored
  , db_instance :: String  -- logical locaton
  } deriving (Show)
  
instance Pretty Location' where
  pretty loc =
    vsep
      [ "Location {"
      , indent 2 ("loc_class         = " <> pretty (loc_class' loc))
      , indent 3 ("loc_name         = " <> pretty (loc_name' loc))
      , indent 3 ("loc_directory    = " <> pretty (loc_directory' loc))     
      , indent 3 ("loc_remote_node  = " <> pretty (loc_remote_node' loc))
      , indent 3 ("loc_remote_login = " <> pretty (loc_remote_login' loc))
      , indent 3 ("loc_remote_port  = " <> pretty (loc_remote_port' loc))
      , indent 3 ("loc_db_user      = " <> pretty (loc_db_user' loc))
      , indent 3 ("loc_description  = " <> pretty (loc_description' loc))
      , indent 3 ("db_node_name     = " <> pretty (db_node_name loc))
      , indent 3 ("db_instance      = " <> pretty (db_instance loc))
      , "}"
      ]  

loc2Location :: Location -> String -> [String] -> Location'
loc2Location x classSuffix [db_user,node_name,db_instance] = 
                              Location' {
                                          loc_name'  = loc_name x 
                                         , loc_class' = loc_class x ++ classSuffix
                                         , loc_directory' = loc_directory x
                                         , loc_remote_node'  = loc_remote_node x
                                         , loc_remote_login' = loc_remote_login x
                                         , loc_remote_port'  = loc_remote_port x
                                         , loc_db_user'      = db_user                  ---- is it agent user or db user?
                                         , loc_description'  = loc_description x
                                         , db_node_name      = node_name
                                         , db_instance       = db_instance
                                         }
                                         

In [3]:
import Text.Read (readMaybe) 
import Data.Maybe (fromMaybe )
import Data.List.Split (splitOn)
import Data.List (intercalate)

parseRedshift :: Location  ->[String]
parseRedshift l  = getUSer l : (concat . filterCols . fromMaybe []. parseJson .  getJsonStr)  l  
   where
       getJsonStr  = drop 1 . dropWhile (/= '=') . loc_db_name  -- get loc_db_name and return string after first '='
       parseJson s = readMaybe s :: Maybe [[String]]            -- parse to list
       filterCols  = map tail .  filter (\[k, _] -> k `elem` ["db_node", "db_name"])
       getUSer = head . splitOn "/" . loc_db_user 
              
parseLoc :: Location  -> Location'
parseLoc l
    | loc_class l == "redshift"   = loc2Location l ""  $ parseRedshift l
    | loc_class l == "file" && ((head . loc_directory)   l  == '/'    )  = loc2Location l "_locdir"  [loc_remote_login  l ,loc_remote_node l, loc_directory l]
    | loc_class l == "file" && ((take 4 . loc_directory) l  == "s3s:" )  = loc2Location l "_s3s" ["###",getBacket l ,getPrefix l]
    | loc_class l == "file" && ((take 5 . loc_directory) l  == "sftp:")  = loc2Location l "_sftp" ["###",getBacket l ,getsftpFolder l]
    | loc_class l `elem` ["mysql","greenplum","postgresql"]              = loc2Location l ""   [ getUserMysql l , getNodeMysql l, getInstMysql l]
    | loc_class l == "sqlserver"    = loc2Location l ""  [getUserMsSql l, getNodeMsSql l,getInstMsSql l ]   
    | loc_class l == "teradata"     = loc2Location l ""   [getUserTd l, loc_db_name l,loc_db_name l ] 
    | loc_class l == "salesforce"   = loc2Location l ""   [getUserSf l, getNodeSf l,getInstSf l ] 
    | otherwise   = loc2Location l ""  ["###","###","###"]   
    where 
    getBacket = head . splitOn "/" . last. splitOn "@" .loc_directory
    getPrefix =  intercalate  "/" .tail . splitOn "/" . last. splitOn "@" .loc_directory
    getsftpNode   = getBacket
    getsftpFolder =  getPrefix
    --
    getNodeMysql  =  head . splitOn "~" . loc_db_name
    getInstMysql  =  last . splitOn "~" . loc_db_name
    getUserMysql  =  head . splitOn "/" . loc_db_user
    --
    getNodeMsSql  =  head . splitOn "\\" . loc_db_name
    getInstMsSql  =  last . splitOn "\\" . loc_db_name
    getUserMsSql  =  getUserMysql
    --
    getUserTd     =  getUserMysql
    --
    getNodeSf     = last .splitOn "//" . loc_directory
    getInstSf     = last .splitOn "@" . head . splitOn "/" . loc_db_user
    getUserSf     = head .splitOn "@" . head . splitOn "/" . loc_db_user

------------

In [ ]:
(last . splitOn "=" . loc_db_name )  $ lll!!1
(last . splitOn "=" . loc_db_name )  $ ll!!1

"[[\"urls\",[\"b-1.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-2.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-3.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\"]],[\"broker_ca\",\"/data/kafka_certs/MSKPOCKey.pem\"],[\"ssl_cert\",\"/data/kafka_devcerts/kafka/signed-certificate-from-acm\"],[\"ssl_key\",\"/data/kafka_devcerts/kafka/key.pem\"],[\"ssl_key_pwd\",\"!{Itbo2aix.D3x7/Yp}!\"]]"

"[[\"odbc_driver\",\"/opt/amazon/redshiftodbc/lib/64/libamazonredshiftodbc64.so\"],[\"db_node\",\"us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com\"],[\"db_port\",\"5439\"],[\"db_name\",\"redshift_hvr\"]]"

In [3]:
-------------

In [4]:
import Data.Either
l = fromRight [] parsedData
length l

299

In [ ]:
ll

[Location {loc_name = "kafka", loc_class = "kafka", loc_directory = "", loc_remote_node = "", loc_remote_login = "", loc_remote_pwd = "", loc_remote_port = 0, loc_db_name = "JSON=[[\"urls\",[\"b-2.odp-us-innovation-msk.zov259.c1.kafka.us-east-1.amazonaws.com:9092\",\"b-1.odp-us-innovation-msk.zov259.c1.kafka.us-east-1.amazonaws.com:9092\",\"b-3.odp-us-innovation-msk.zov259.c1.kafka.us-east-1.amazonaws.com:9092\"]],[\"default_topic\",\"{hvr_tbl_name}\"]]", loc_db_user = "", loc_description = ""},Location {loc_name = "kafk1", loc_class = "kafka", loc_directory = "", loc_remote_node = "10.242.109.196", loc_remote_login = "hvr", loc_remote_pwd = "!{lx5b1s4XljZxBFYJ}!", loc_remote_port = 4343, loc_db_name = "JSON=[[\"urls\",[\"b-1.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-2.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\",\"b-3.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094\"]],[\"broker_ca\",\"/data/kafka_certs/MSKPOCKey.pem\"],

In [8]:
-- helper function to get 
ll =  filter (\x ->  loc_class x  == "kafka" ) l
lll =  filter (\x ->  loc_class x  == "kafka" ) l
map pretty    ll

[Location {
  loc_name = kafka
  loc_class = kafka
  loc_directory = 
  loc_remote_node = 
  loc_remote_login = 
  loc_remote_pwd = 
  loc_remote_port = 0
  loc_db_name = JSON=[["urls",["b-2.odp-us-innovation-msk.zov259.c1.kafka.us-east-1.amazonaws.com:9092","b-1.odp-us-innovation-msk.zov259.c1.kafka.us-east-1.amazonaws.com:9092","b-3.odp-us-innovation-msk.zov259.c1.kafka.us-east-1.amazonaws.com:9092"]],["default_topic","{hvr_tbl_name}"]]
  loc_db_user = 
  loc_description = 
},Location {
  loc_name = kafk1
  loc_class = kafka
  loc_directory = 
  loc_remote_node = 10.242.109.196
  loc_remote_login = hvr
  loc_remote_pwd = !{lx5b1s4XljZxBFYJ}!
  loc_remote_port = 4343
  loc_db_name = JSON=[["urls",["b-1.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094","b-2.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094","b-3.us-dev-msk-cluster.x8853p.c1.kafka.us-east-1.amazonaws.com:9094"]],["broker_ca","/data/kafka_certs/MSKPOCKey.pem"],["ssl_cert","/data/kafka_devcert

In [9]:
--pretty $ l!!6
--loc2Location (l!!6) "_RED" ["db_user","node_name","node_instance"]
--loc2Location (l!!6) "_RED"  $ parseRedshift (l!!6)

In [10]:
--l6 =  l!!6
--pretty l6
--l6' = loc2Location l6 "_RED" ["db_user","node_name","node_instance"]
--pretty l6'

In [10]:
l' = map  parseLoc l

In [19]:
 map pretty (filter (\x ->  loc_class' x == "redshift" ) l' )

[Location {
  loc_class         = redshift
   loc_name         = dsred
   loc_directory    = 
   loc_remote_node  = 10.242.112.153
   loc_remote_login = hvr2
   loc_remote_port  = 9090
   loc_db_user      = 502830612
   loc_description  = Redshift data sharing cluster
   db_node_name     = redshift-data-shared-poc.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com
   db_instance      = redshiftpocdb
},Location {
  loc_class         = redshift
   loc_name         = redst
   loc_directory    = 
   loc_remote_node  = 10.242.112.153
   loc_remote_login = gehc_hvr
   loc_remote_port  = 4343
   loc_db_user      = redshiftuser
   loc_description  = redshift development
   db_node_name     = us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com
   db_instance      = redshift_hvr
},Location {
  loc_class         = redshift
   loc_name         = rdite
   loc_directory    = 
   loc_remote_node  = 10.242.112.153
   loc_remote_login = hvr2
   loc_remote_port  = 9090
   loc_db_user      = 